In [1]:
import os
import glob
import jieba
import spacy
import opencc
import string
import gensim
import requests
import fasttext
import fasttext.util
import zh_core_web_trf
import spacy_transformers

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from stopwordsiso import stopwords
from gensim.test.utils import datapath
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.pipeline.textcat_multilabel import Config, multi_label_cnn_config

In [2]:
os.chdir('/Users/shenchingfeng/Desktop/Topic Modeling/')

In [27]:
df = pd.read_csv('./chat_df_w_file_usr_time_text.csv')
df_emoji = pd.read_csv('./df_emoji.csv', encoding = 'utf-8')
df_remove_emoji = pd.read_csv('./df_remove_emoji.csv', encoding = 'utf-8')
df_emoji_to_desc = pd.read_csv('./df_emoji_to_desc.csv', encoding = 'utf-8')

In [86]:
stop_words_combined = list(stopwords(["zh"]))

cc = opencc.OpenCC('s2t.json')
stopword = []

for i in stop_words_combined:
    stopword.append(cc.convert(i))

In [7]:
jieba.load_userdict('./dict.txt.big')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/56/1hl93h654354jd5vh2zjmd640000gn/T/jieba.cache
Loading model cost 0.458 seconds.
Prefix dict has been built successfully.


In [122]:
def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation + "！？&#8203;``【oaicite:0】``&#8203;（）［］《》、，。；：‘“’”…￥·")
    text_without_punct = text.translate(translator)
    return text_without_punct

def jieba_cut(text):
    if pd.isna(text):
        return []

    text_without_punct = remove_punctuation(text)
    seg_list = [seg for seg in jieba.cut(text_without_punct) if seg.strip()]
    
    return seg_list

def tokenizer(doc):

    seg = jieba_cut(doc)
    filtered_seg = [word for word in seg if(word not in stopword) and (len(word) > 1)]

    return filtered_seg

In [168]:
final = []
for index, row in df_remove_emoji.iterrows():
    segments = tokenizer(row['text'])
    final.append(segments)

In [217]:
os.chdir('./word2vec/df_1209/')

In [218]:
model = Word2Vec.load('./1209_w2v_try.bin')

In [405]:
keyword = '裁判'

res = model.wv.similar_by_word(keyword, topn = 100)
for item in res:
    print(item)

('聊天室', 0.9174688458442688)
('她', 0.9076641798019409)
('用', 0.9059139490127563)
('不用', 0.9002881050109863)
('這樣', 0.8989401459693909)
('觸網', 0.89765864610672)
('自己', 0.895721971988678)
('鄧公', 0.8947786092758179)
('只是', 0.8936440348625183)
('連董', 0.8910359740257263)
('他', 0.884768545627594)
('時間', 0.8831663131713867)
('主審', 0.883057713508606)
('還要', 0.8780483603477478)
('不然', 0.876921534538269)
('先', 0.8766741752624512)
('跑', 0.8761611580848694)
('他們', 0.8760174512863159)
('結果', 0.8752740621566772)
('問題', 0.8745262026786804)
('XD', 0.8745228052139282)
('直播', 0.874515175819397)
('暫停', 0.8738765716552734)
('挑戰', 0.8737232685089111)
('一直', 0.8729910850524902)
('哪', 0.8714717626571655)
('XDD', 0.8713074326515198)
('昭銘', 0.8708035349845886)
('國家隊', 0.8691540360450745)
('哪裡', 0.8689779043197632)
('有人', 0.8666054606437683)
('當', 0.8665590286254883)
('講', 0.8660700917243958)
('球迷', 0.8652830123901367)
('人家', 0.8645651340484619)
('這種', 0.864547848701477)
('那邊', 0.8630650043487549)
('選手', 0.86286

In [484]:
team_emoji = ['☀', '⚡', '🌚', '🌝', '🌞', '🐳', '🐋', '🍊', '🏎', '🔆', '🐯', '🦁']
cheer_emoji = ['⛽', '🏐', '💓', '💕', '💖', '💗', '💘', '💙', '💚', '💛', '🏆', '👑', '🎉', '🎊', '❤', '🦾', '💯', '🙏', '💪', '🔥', '😍', '🤩']

In [540]:
os.chdir('/Users/shenchingfeng/Desktop/Topic Modeling/team_player/')

In [541]:
file = glob.glob('./*.txt')

player_name = []
for t in file:
    with open(t, 'r') as f:
        names = [name.strip() for name in f.readlines()]
        player_name.extend(names)

In [539]:
os.chdir('/Users/shenchingfeng/Desktop/Topic Modeling/')

In [495]:
team_file = './team.txt'
team = []
with open(team_file, 'r') as f:
    team_name = [t.strip() for t in f.readlines()]

team_name.append('球隊')

In [542]:
cheer_words = ['加油', '恭喜', '最棒', '穩住', '撐住', '一鼓作氣', '棒棒', '衝啊', '再接再厲', '追分', '沒事', '別氣餒', '不要氣餒', '保持', '穩穩', '咬住', '穩贏', 'GOGOGO', '無敵', '777', '666', '緊張', '爽', '啊啊', '刺激', '哇靠', '喔喔', '精彩', '好球', '水啦', '水唷', '來了', 'OMG', '厲害', '很棒', '哈哈', 'ha', 'HA', 'Ha', '激動', '狂', '神', '太強', '帥', '送啦', '好看', '讚', '好猛', '可惜', '耶', '超鬼', 'lucky', 'Lucky', 'Wow', 'wow', 'WOW', 'yey', 'yes', 'YEY', 'Yey', 'Yes', 'YES', '+u', '贏球', '水喔', '氣勢', '猛', '感動', '贏']
tactic_words = ['攔網', '快攻', '進攻', '后排', '後排', '前排', '發球', '攻擊', '防守', '接發', '救', '接球', '到位', '接發球', '發球', '攔', '舉球', '一傳', '角度', '主攻', '二傳', '輪轉', 'block', 'IN', 'in', 'In', '界內', '封網', '失誤', '觸球', '暫停', '換人', '落地', '得分', '舉', '界外', 'outside', '局末', 'ACE', 'deuce', 'ace', 'Deuce', 'deu', '吊球', '短球', 'touch', '平手', '觸網', '自由', '連擊', '欄網']
coach_words = ['連董', '小牛', '牛', '鄧公', '教練', '教练', '舟']
stream_words = ['導播', '小編', '球評', '轉播', '講評', '聊天室', '畫面', '直播', '致平', '大康', '畫質', '邱老師', '電視', '邱老师', '000人', '000 人', '000', '千人', 'HOP SPORTS', '鏡頭', '攝影機', 'HOP Sports', '看球', '人數', '視角', '重播', '運鏡', '鏡位', '延遲', '祐哥']
judge_words = ['裁判', '誤判', '挑戰', '規則', '主播', '吹', '鷹眼', '黃牌', '抗議', '判']

In [555]:
## 加油 
def contains_cheer_emoji(text): ## 3890
    return any(e in text for e in cheer_emoji)
def contains_cheer_word(text):  ## 27990
    return any(word in text for word in cheer_words)

## 教練
def contains_coach_word(text): ## 3211
    return any(word in text for word in coach_words)

## 戰術
def contains_tactic_word(text): ## 14667
    return any(word in text for word in tactic_words)

## 球員
def contains_player_name(text): ## 26534
    return any(word in text for word in player_name)

## 球隊
def contains_team_name(text):  ## 18078
    return any(word in text for word in team_name)
def contains_team_emoji(text): ## 5007
    return any(word in text for word in team_emoji)

## 轉播
def contains_stream_word(text):  ## 2684
    return any(word in text for word in stream_words)

## 裁判
def contains_judge_word(text):  ## 2746
    return any(word in text for word in judge_words)

In [545]:
len(df[df['text'].apply(contains_coach_word)])

3211

In [547]:
no_condition_rows = df[
    (~df['text'].apply(contains_cheer_emoji)) &
    (~df['text'].apply(contains_cheer_word)) &
    (~df['text'].apply(contains_coach_word)) &
    (~df['text'].apply(contains_tactic_word)) &
    (~df['text'].apply(contains_team_emoji)) &
    (~df['text'].apply(contains_player_name)) &
    (~df['text'].apply(contains_team_name)) & 
    (~df['text'].apply(contains_stream_word)) & 
    (~df['text'].apply(contains_judge_word))
]

count_no_condition_rows = len(no_condition_rows)

print(f"Number of rows not meeting any condition: {count_no_condition_rows}") ## 46959

Number of rows not meeting any condition: 46959


### 比賽 / 加油 / 轉播 / 其他

In [563]:
df_h1 = df.copy()

In [564]:
df_h1['比賽'] = ''
df_h1['加油'] = ''
df_h1['轉播'] = ''
df_h1['球迷的聊天'] = ''

for i, row in df_h1.iterrows():

    if contains_cheer_emoji(row['text']) or contains_cheer_word(row['text']):
        df_h1.at[i, '加油'] = 1
    else:
        df_h1.at[i, '加油'] = 0
    
    if contains_coach_word(row['text']) or contains_player_name(row['text']) or contains_tactic_word(row['text']) or contains_team_name(row['text']) or contains_team_emoji(row['text']) or contains_judge_word(row['text']):
        df_h1.at[i, '比賽'] = 1
    else:
        df_h1.at[i, '比賽'] = 0

    if contains_stream_word(row['text']):
        df_h1.at[i, '轉播'] = 1
    else:
        df_h1.at[i, '轉播'] = 0
    
    if not (contains_cheer_emoji(row['text']) or contains_cheer_word(row['text']) or contains_coach_word(row['text']) or contains_player_name(row['text']) or contains_tactic_word(row['text']) or contains_team_name(row['text']) or contains_team_emoji(row['text']) or contains_judge_word(row['text']) or contains_stream_word(row['text'])):
        df_h1.at[i, '球迷的聊天'] = 1
    else:
        df_h1.at[i, '球迷的聊天'] = 0

In [568]:
y = df_h1[['比賽', '加油', '轉播', '球迷的聊天']]
y = y.to_dict('index')

In [572]:
dataset = list(zip(df_h1['text'],[{'cats': cats} for cats in y.values()]))
print(dataset[0])

('小温加油！美津濃加油🔥🔥', {'cats': {'比賽': 1, '加油': 1, '轉播': 0, '球迷的聊天': 0}})


In [573]:
train_data, test_data = train_test_split(dataset, train_size = 0.8, random_state = 42)

In [585]:
# nlp = spacy.load('zh_core_web_trf')
nlp = zh_core_web_trf.load()

# config = Config().from_str(multi_label_cnn_config)

# text_cat = nlp.add_pipe("textcat_multilabel", config = config)

# labels = ["比賽", "加油", "轉播", "球迷的聊天"]

# text_cat.add_labels(labels)

ValueError: [E002] Can't find factory for 'curated_transformer' for language Chinese (zh). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, transformer

In [ ]:
for epoch in range(10):
    for text, annotations in train_data:
        doc = nlp(text)
        example = spacy.training.Example.from_dict(doc, annotations)
        text_cat.update([example])

nlp.to_disk("multi_label_model_zh")

In [69]:
url = 'https://tvl.ctvba.org.tw/news'
response = requests.get(url)
news = {'URL': [], 'News': []}

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    news_links = soup.find_all('a', href = lambda href: href and "/news-detail/" in href)

    for link in news_links:
        news_url = 'https://tvl.ctvba.org.tw' + link['href']
        news_response = requests.get(news_url)

        if news_response.status_code == 200:
            news_soup = BeautifulSoup(news_response.text, 'html.parser')
            
            description_meta = news_soup.find('meta', {'name': 'description'})

            if description_meta:
                description_content = description_meta['content']

                news['URL'].append(news_url)
                news['News'].append(description_content)
else:
    print('Failed to retrieve the page. Status Code:', response.status_code)

news = pd.DataFrame(news)

In [215]:
# news = news.drop_duplicates(subset = ['News']).reset_index(drop = True)
# news['News'] = news['News'].str.replace('\r\n', '')
# news['News'] = news['News'].str.replace('\xa0', '')
# news = news[:307]
# news

In [123]:
# news_final = []

# for index, row in news.iterrows():
#     segments = tokenizer(row['News'])
#     news_final.append(segments)

In [135]:
# all = final + news_final
# all_filtered = [sublist for sublist in all if len(sublist) >= 1]

In [171]:
# r = [' '.join(rrr) for rrr in all_filtered]

In [182]:
# model = Word2Vec(all_filtered, vector_size = 300, min_count = 3, workers = -1, sg = 1)
# model.save("1209W2V_News_and_Chat.model")

In [209]:
# keyword = 'Carla'

# res = word2vec_model.wv.similar_by_word(keyword, topn = 10)
# for item in res:
#     print(item)

('還糟', 0.279350221157074)
('停車位', 0.2471543848514557)
('waaaaahhhh', 0.244615837931633)
('短期內', 0.24159204959869385)
('分部', 0.22809460759162903)
('達叔', 0.22177664935588837)
('看不太到', 0.21698904037475586)
('看西米', 0.21578575670719147)
('看小姿', 0.21564465761184692)
('非同小可', 0.21360906958580017)


In [190]:
# text_list = df_remove_emoji['text'].tolist()
# news_list = news['News'].tolist()
# documents = text_list + news_list

In [204]:
# import jieba
# from sklearn.feature_extraction.text import TfidfVectorizer
# from gensim.models import Word2Vec
# import numpy as np

# documents = [str(doc) for doc in documents]

# # Step 1: Tokenize using Jieba and compute TF-IDF vectors
# jieba_documents = [' '.join(jieba.cut(doc)) for doc in documents]
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(jieba_documents)

# # Step 2: Train Word2Vec model
# word2vec_model = Word2Vec(sentences=[jieba.lcut(doc) for doc in documents], vector_size = 300, window = 5, min_count = 1, workers = -1)

# # Step 3: Weight Word2Vec vectors with TF-IDF scores
# weighted_word_vectors = []
# for i, doc in enumerate(documents):
#     tfidf_scores = {word: tfidf_matrix[i, tfidf_vectorizer.vocabulary_[word]] for word in jieba.lcut(doc) if word in tfidf_vectorizer.vocabulary_}
#     weighted_vectors = [word2vec_model.wv[word] * tfidf_scores[word] for word in tfidf_scores]
#     if weighted_vectors:
#         weighted_doc_vector = np.mean(weighted_vectors, axis=0)
#         weighted_word_vectors.append(weighted_doc_vector)
#     else:
#         weighted_word_vectors.append(np.zeros(word2vec_model.vector_size))

In [206]:
# from sklearn.metrics.pairwise import cosine_similarity
# similarity_matrix = cosine_similarity(weighted_word_vectors[:2], weighted_word_vectors[:2])
# print("Cosine Similarity Matrix:")
# print(similarity_matrix)

Cosine Similarity Matrix:
[[0.9999997 0.6943898]
 [0.6943898 1.       ]]
